<a href="https://colab.research.google.com/github/tntly/ml-projects/blob/main/wine_quality_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [61]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import TensorDataset, DataLoader

In [20]:
wine = pd.read_csv('/content/drive/MyDrive/CS 271/Assignments/wine_quality.csv')

wine.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,8.291114,0.486442,0.300503,0.900000,0.015739,10.119501,83.835280,0.997348,3.390186,0.654920,8.809257,Medium-High
1,8.750731,0.548988,0.264489,2.257949,0.114811,24.012219,6.000000,0.995788,3.320167,0.552764,9.220956,Medium-High
2,7.121011,0.416757,0.171422,2.447964,0.069355,5.883556,54.381763,0.995753,3.067723,0.675910,10.867600,Medium
3,11.325877,0.397869,0.008664,2.483803,0.156098,23.928658,98.454793,0.998876,3.399212,0.930702,13.376553,Medium
4,6.806398,0.597507,0.365772,3.219262,0.021520,1.000000,44.882639,0.995727,3.172473,0.863165,10.244412,Medium-High


In [21]:
wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         5000 non-null   float64
 1   volatile acidity      5000 non-null   float64
 2   citric acid           5000 non-null   float64
 3   residual sugar        5000 non-null   float64
 4   chlorides             5000 non-null   float64
 5   free sulfur dioxide   5000 non-null   float64
 6   total sulfur dioxide  5000 non-null   float64
 7   density               5000 non-null   float64
 8   pH                    5000 non-null   float64
 9   sulphates             5000 non-null   float64
 10  alcohol               5000 non-null   float64
 11  quality               5000 non-null   object 
dtypes: float64(11), object(1)
memory usage: 468.9+ KB


In [22]:
wine.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,8.349835,0.526379,0.276535,2.594706,0.088236,16.260726,48.690879,0.996768,3.311366,0.658122,10.461020
std,1.717194,0.176106,0.183769,1.273142,0.044430,9.735590,30.425146,0.001856,0.155415,0.166482,1.033777
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000
25%,7.195541,0.404292,0.132661,1.539660,0.055164,8.892291,23.982949,0.995522,3.204367,0.540422,9.737473
50%,8.337575,0.526369,0.264508,2.510346,0.086996,16.030426,47.035699,0.996744,3.312448,0.657467,10.426220
75%,9.520339,0.646922,0.403052,3.469872,0.118883,22.933460,69.755513,0.998018,3.415951,0.770503,11.159249
max,15.417004,1.198324,0.936551,7.994076,0.251429,53.003917,163.287601,1.004006,3.836557,1.245648,14.370318


In [23]:
wine["quality"].value_counts()

,count
quality,
Medium-Low,2151
Medium,2016
Medium-High,620
Low,163
High,50


In [25]:
# Define custom mapping
quality_mapping = {
    "Low": 0,
    "Medium-Low": 1,
    "Medium": 2,
    "Medium-High": 3,
    "High": 4
}

# Apply the mapping to the quality column
wine["quality_encoded"] = wine["quality"].map(quality_mapping)

wine.head()


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,quality_encoded
0,8.291114,0.486442,0.300503,0.900000,0.015739,10.119501,83.835280,0.997348,3.390186,0.654920,8.809257,Medium-High,3
1,8.750731,0.548988,0.264489,2.257949,0.114811,24.012219,6.000000,0.995788,3.320167,0.552764,9.220956,Medium-High,3
2,7.121011,0.416757,0.171422,2.447964,0.069355,5.883556,54.381763,0.995753,3.067723,0.675910,10.867600,Medium,2
3,11.325877,0.397869,0.008664,2.483803,0.156098,23.928658,98.454793,0.998876,3.399212,0.930702,13.376553,Medium,2
4,6.806398,0.597507,0.365772,3.219262,0.021520,1.000000,44.882639,0.995727,3.172473,0.863165,10.244412,Medium-High,3


In [43]:
# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler on X (features) and transform X to normalize it
X = scaler.fit_transform(wine.drop(columns=["quality", "quality_encoded"]))

y = wine["quality_encoded"]

In [55]:
# Split the dataset into training and test sets (80% training, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [59]:
# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)  # Ensure targets are long for classification

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)  # Ensure targets are long for classification

In [ ]:
# Create TensorDataset
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create DataLoader for batch processing
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)